# Recurrent Neural Networks
You should build an end-to-end machine learning pipeline using a recurrent neural network model. In particular, you should do the following:
- Load the `jena climate` dataset using [Pandas](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html). You can find this dataset in the [keras repository](https://keras.io/examples/timeseries/timeseries_weather_forecasting/).
- Split the dataset into training, validation, and test sets. Note that you cannot split time series using [Scikit-Learn](https://keras.io/examples/timeseries/timeseries_weather_forecasting/).
- Build an end-to-end machine learning pipeline, including a [recurrent neural network](https://keras.io/examples/timeseries/timeseries_weather_forecasting/) model.
- Optimize your pipeline by validating your design decisions.
- Test the best pipeline on the test set and report various [evaluation metrics](https://scikit-learn.org/0.15/modules/model_evaluation.html).  
- Check the documentation to identify the most important hyperparameters, attributes, and methods of the model. Use them in practice.

1. Load the data
2. Take a sample from the dataset. select measurements are recorded at 12pm.
3. Split the data into training, validation, and test sets.
4. Creating sequences
5. Design and Train your model
6. Test your model

Importing Libraries

In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

Reading the Dataset

In [31]:
df = pd.read_csv('/content/jena_climate_2009_2016.csv')
df.shape

(420551, 15)

EDA

In [32]:
df.head()

,Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
0,01.01.2009 00:10:00,996.52,-8.02,265.40,-8.90,93.3,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
1,01.01.2009 00:20:00,996.57,-8.41,265.01,-9.28,93.4,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1
2,01.01.2009 00:30:00,996.53,-8.51,264.91,-9.31,93.9,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6
3,01.01.2009 00:40:00,996.51,-8.31,265.12,-9.07,94.2,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0
4,01.01.2009 00:50:00,996.51,-8.27,265.15,-9.04,94.1,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3


In [33]:
df.isnull().sum()

,0
Date Time,0
p (mbar),0
T (degC),0
Tpot (K),0
Tdew (degC),0
rh (%),0
VPmax (mbar),0
VPact (mbar),0
VPdef (mbar),0
sh (g/kg),0


In [34]:
df['Date Time'] = pd.to_datetime(df['Date Time'], format='%d.%m.%Y %H:%M:%S')

df_12pm = df[df['Date Time'].dt.time == pd.to_datetime('12:00:00').time()]

df_12pm = df_12pm.sort_values('Date Time')

df_12pm = df_12pm.sort_values('Date Time').reset_index(drop=True)

In [56]:
df_12pm.head(3)

,Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
0,2009-01-01 12:00:00,1000.30,-6.87,266.27,-8.28,89.6,3.64,3.27,0.38,2.03,3.26,1306.98,1.84,2.63,184.4
1,2009-01-02 12:00:00,998.91,-3.12,270.13,-4.50,90.1,4.85,4.37,0.48,2.73,4.38,1286.47,1.54,2.00,127.0
2,2009-01-03 12:00:00,999.02,-5.96,267.28,-8.72,80.7,3.91,3.16,0.75,1.97,3.16,1300.90,1.31,1.88,213.9


In [35]:
df_12pm.shape

(2921, 15)

Data Splitting

In [36]:
n = len(df_12pm)
train_end = int(n * 0.7)
val_end = int(n * 0.85)

train_df = df_12pm[:train_end]
val_df = df_12pm[train_end:val_end]
test_df = df_12pm[val_end:]

In [37]:
print("Train size:", len(train_df))
print("Validation size:", len(val_df))
print("Test size:", len(test_df))

Train size: 2044
Validation size: 438
Test size: 439


Data Preprocessing

In [38]:
features = [
    'p (mbar)', 'Tpot (K)', 'Tdew (degC)', 'rh (%)', 'VPmax (mbar)', 'VPact (mbar)',
    'VPdef (mbar)', 'sh (g/kg)', 'H2OC (mmol/mol)', 'rho (g/m**3)', 'wv (m/s)',
    'max. wv (m/s)', 'wd (deg)'
]
target = 'T (degC)'

In [39]:
X_train = train_df[features].values
y_train = train_df[target].values

X_val = val_df[features].values
y_val = val_df[target].values

X_test = test_df[features].values
y_test = test_df[target].values

Feature Engineering

In [40]:
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_val_scaled = scaler_X.transform(X_val)
X_test_scaled = scaler_X.transform(X_test)

scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1,1))
y_val_scaled = scaler_y.transform(y_val.reshape(-1,1))
y_test_scaled = scaler_y.transform(y_test.reshape(-1,1))

Creating Sequence

In [43]:
def create_sequences(X, y, seq_length=5):
    X_seq, y_seq = [], []
    for i in range(len(X) - seq_length):
        X_seq.append(X[i:i+seq_length])
        y_seq.append(y[i+seq_length])
    return np.array(X_seq), np.array(y_seq)

seq_length = 5

In [44]:
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, seq_length)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, seq_length)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, seq_length)

print("Train sequence shape:", X_train_seq.shape)
print("Validation sequence shape:", X_val_seq.shape)
print("Test sequence shape:", X_test_seq.shape)

Train sequence shape: (2039, 5, 13)
Validation sequence shape: (433, 5, 13)
Test sequence shape: (434, 5, 13)


Model Training

In [46]:
model = Sequential([
    LSTM(64, activation='tanh', input_shape=(seq_length, len(features))),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [47]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=100,
    batch_size=16,
    callbacks=[early_stop]
)

Epoch 1/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 0.4201 - mae: 0.4806 - val_loss: 0.1648 - val_mae: 0.3199
Epoch 2/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1599 - mae: 0.3150 - val_loss: 0.1595 - val_mae: 0.3148
Epoch 3/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1440 - mae: 0.2996 - val_loss: 0.1461 - val_mae: 0.2990
Epoch 4/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1362 - mae: 0.2910 - val_loss: 0.1456 - val_mae: 0.2994
Epoch 5/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1498 - mae: 0.2984 - val_loss: 0.1412 - val_mae: 0.2922
Epoch 6/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1322 - mae: 0.2830 - val_loss: 0.1422 - val_mae: 0.2925
Epoch 7/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1194 - mae: 0.2701 - val_loss: 0.1371 - val_mae: 0.2869
Epoch 8/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1305 - mae: 0.2807 - val_loss: 0.1349 - val_mae: 0.2882
Epoch 9/100
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms

In [53]:
test_loss, test_mae = model.evaluate(X_test_seq, y_test_seq)
print(f"Test MAE: {test_mae*100:.4f}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.1169 - mae: 0.2741
Test MAE: 25.8516
